# Good Programming Habits
* Check the inputs
* Test incrementally...
* Prefer to use pure functions
* Prefer List to Tuple unless you really need to modify it or need List's features
* Prefer function to scripts
* Pay attention to function naming and write concise doctring to it

# Day 0: Imports and Utility Functions

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import sys
import os
import re
import numpy as np
import random
import collections, functools, itertools, operator
from collections import Counter, defaultdict, namedtuple, deque, OrderedDict
from functools   import lru_cache, reduce
from statistics  import mean, median, mode, stdev, variance
from itertools   import (permutations, combinations, groupby, cycle, chain, 
                         zip_longest, takewhile, islice, dropwhile, count as count_from)
from heapq       import heappush, heappop
from operator    import iand, ior, ilshift, irshift, ixor
from numba       import jit

In [2]:
origin = (0, 0)
HEADINGS = UP, LEFT, DOWN, RIGHT = (0, -1), (-1, 0), (0, 1), (1, 0)

def first(iterable, default=None):
    "return the first item in an iterable, or default if it is empty"
    return next(iter(iterable), default)

def nth(iterable, n, default=None):
    "return the nth item of the iterable or a default value"
    return next(islice(iterable, n, None), default)

def neighbors4(P):
    "yield four neighbors of point P"
    x, y = P
    for dx, dy in (-1, 0), (1, 0), (0, 1), (0, -1):
        yield (x + dx, y + dy)

def neighbors8(P):
    "yield eight neighbors of point P"
    x, y = P
    for dx in (-1, 0, 1):
        for dy in (-1, 0, 1):
            if dx == dy == 0:
                continue
            yield (x + dx, y + dy)

def cityblock_distance(P, Q=origin):
    "Manhatten distance between two points"
    return sum(abs(p - q) for p, q in zip(P, Q))


# [Day 1: Inverse Captcha](https://adventofcode.com/2017/day/1)

1. Check backwards instead of using % operator
2. Write List comprehension in multiple lines to enhance readbility.

In [3]:
# Part 1. sum of all digits that match the next digit in the list
# Part 2. consider the digit halfway around the circular
with open('inputs/day1.txt') as f:
    line = f.read()
    line = tuple(map(int, line))
    n, half = len(line), len(line) // 2
    print(sum(line[i]
          for i in range(n)
          if line[i] == line[i - 1])) # check backwards
    print(sum(line[i]
          for i in range(n)
          if line[i] == line[i - half]))

1069
1268


# [Day 2: Corruption Checksum](https://adventofcode.com/2017/day/2)

1. Using numpy loadtxt to read file, do not forget to set dtype

In [4]:
# 1. sum of difference of each row
m = np.loadtxt('inputs/day2.txt', dtype=np.int64)

print(sum(row.max() - row.min() for row in m))

# Part 1
# method 1
print(sum([max(a, b) // min(a, b)
           for a, b in combinations(row, 2)
           if max(a, b) % min(a, b) == 0][0]
           for row in m))

# method 2
def evendiv(row):
    return first(a // b for a in row for b in row if a > b and a % b == 0)
print(sum(map(evendiv, m)))

45158
294
294


# [Day 3: Spiral Memory](https://adventofcode.com/2017/day/3)

1. Spiral patterns: increase side gradually
2. Use RIGHT, UP, LEFT, DOWN and neighbors8()
3. Use generator

In [5]:
def spiral():
    "Yield successive (x, y) coordinates of squares on a spiral"
    x = y = side = 0
    yield(x, y)
    while True:
        for dx, dy in (RIGHT, UP, LEFT, DOWN):
            if dx: # need to increment side before LEFT and RIGHT
                side += 1
            for _ in range(side):
                x, y = x + dx, y + dy
                yield (x, y)
print(list(islice(spiral(), 4)))

[(0, 0), (1, 0), (1, -1), (0, -1)]


In [6]:
n = 277678
pos = nth(spiral(), n - 1)
print(cityblock_distance(pos))

475


In [7]:
def spiralsums():
    "yield the values of a spiral where each square has the sum of the 8 neighbors"
    values = defaultdict(int)
    for p in spiral():
        values[p] = sum(values[nei] for nei in neighbors8(p)) or 1
        yield values[p]

In [8]:
list(islice(spiralsums(), 10))

[1, 1, 2, 4, 5, 10, 11, 23, 25, 26]

In [9]:
print(first(v for v in spiralsums() if v > n))

279138


# [Day 4: High-Entropy Passphrases](https://adventofcode.com/2017/day/4) 

1. Think about program patterns

In [10]:
def is_valid(line):
    # this is faster than len(lst) == len(set(lst))
    s = set()
    for word in line.split():
        if word in s:
            return False
        s.add(word)
    return True

def is_valid_new(line):
    s = set()
    for word in line.split():
        word = ''.join(sorted(word)) # check anagrams
        if word in s:
            return False
        s.add(word)
    return True

with open('inputs/day4.txt', 'r') as f:
    lines = f.readlines()
    print(sum(is_valid(line.strip())
          for line in lines))
    print(sum(is_valid_new(line.strip())
          for line in lines))

451
223


# [Day 5: A Maze of Twisty Trampolines, All Alike](https://adventofcode.com/2017/day/5)

1. variable names: pc, steps, program, memory, run
2. Distinguish between program and memory
3. add verbose optional variable for debug
4. numba @jit to accelerate

In [11]:
with open('inputs/day5.txt', 'r') as f:
    program = list(map(int, f.readlines()))

def run(program, verbose=False):
    memory = list(program)
    pc = 0
    steps = 0
    M = len(memory)
    while 0 <= pc < M:
        jump = memory[pc]
        memory[pc] += 1
        pc += jump
        steps += 1
        if verbose: print(pc, steps, memory)
    return steps

assert(run([0, 3, 0, 1, -3]) == 5)
print(run(program))

336905


In [12]:
@jit
def run2(program, verbose=False):
    memory = list(program)
    pc = 0
    steps = 0
    M = len(memory)
    while 0 <= pc < M:
        jump = memory[pc]
        if jump >= 3:
            memory[pc] -= 1
        else:
            memory[pc] += 1
        pc += jump
        steps += 1
        if verbose: print(pc, steps, memory)
    return steps
    
print(run2(program))

21985262


# [Day 6: Memory Reallocation](https://adventofcode.com/2017/day/6)

1. How to get maximum index?
2. Use count_from instead of idx+= 1 and where

In [13]:
with open('inputs/day6.txt', 'r') as f:
    banks = tuple(map(int, f.read().split()))

In [14]:
def spread(banks):
    "find the bank with with most blocks and spread them to following banks"
    banks = list(banks) # not modify it
    n = len(banks)
    maxi = max(range(n), key=lambda i : banks[i])
    blocks = banks[maxi]
    banks[maxi] = 0
    # method 1
#     q, r = divmod(blocks, n)
#     for i in range(n):
#         banks[i] += q
#     for j in range(maxi + 1, maxi + 1 + r):
#         banks[j % n] += 1
    # method 2 from Peter Norvig, slower but clearer
    for i in range(maxi + 1, maxi + 1 + blocks):
        banks[i % n] += 1
    return tuple(banks)

In [15]:
def realloc(banks):
    "update banks until a repeat pattern is seen, return number of cycles needed"
    seen = {banks}
    for cycles in count_from(1):
        banks = spread(banks)
        if banks in seen:
            return cycles
        seen.add(banks)

In [16]:
assert(realloc((0, 2, 7, 0)) == 5)
print(realloc(banks))

11137


In [17]:
def realloc2(banks):
    "update banks until a repeat pattern is seen, return the length of the cycle"
    seen = {banks : 0}
    for cycles in count_from(1):
        banks = spread(banks)
        if banks in seen:
            return cycles - seen[banks]
        seen[banks] = cycles

In [18]:
assert(realloc2((0, 2, 7, 0)) == 4)
print(realloc2(banks))

1037


# Day 7: Recursive Circus

In [ ]:
nodes = set()
weights = {}
pres, sucs = defaultdict(set), defaultdict(set)
with open('inputs/day7.txt', 'r') as f:
    for line in f:
        line = line.strip()
        idx = line.index('(')
        node = line[:idx - 1]
        weights[node] = int(line[idx + 1:line.index(')', idx)])
        nodes.add(node)
        if '->' in line:
            left, right = line.split(' -> ')
            for nei in right.split(', '):
                nodes.add(nei)
                pres[nei].add(node)
                sucs[node].add(nei)

In [ ]:
nodes - pres.keys()

In [ ]:
def traverse(node):
    cnt = 0
    counter = {}
    for child in sucs[node]:
        temp = traverse(child)
        cnt += temp
        counter[child] = temp
    if len(set(counter.values())) > 1:
        m = mode(counter.values())
        for key, c in counter.items():
            if c != m:
                print(m - c + weights[key])
                cnt += m - c
                break
    return cnt + weights[node]

In [ ]:
# traverse('tknk')
traverse('aapssr');

# Day 8: I Heard You Like Registers

In [ ]:
registers = defaultdict(int)
ops = {'inc' : operator.add, 'dec' : operator.sub}
high = 0
with open('inputs/day8.txt', 'r') as f:
    for line in f:
        lvar, op, num, _, condition = line.split(' ', 4)
        num = int(num)
        rvar, _ = condition.split(' ', 1)
        registers[lvar] = ops[op](registers[lvar], (num if eval(condition, {rvar : registers[rvar]}) else 0))
        high = max(high, registers[lvar])
print(max(registers.values()))
print(high)

# Day 9: Stream Processing

In [19]:
def parse_line(line):
    def helper1(line):
        ans = []
        i = 0
        while i < len(line):
            if line[i] == '!':
                i += 2
            else:
                ans.append(line[i])
                i += 1
        return ''.join(ans)
    def helper2(line):
        nonlocal cnt
        lidx = line.find('<')
        if lidx == -1:
            return line
        else:
            ridx = line.find('>', lidx)
            cnt += ridx - lidx - 1
            return line[:lidx] + helper2(line[ridx + 1:])
    cnt = 0
    line = helper1(line)
    line = helper2(line)
    print(cnt)
    return line
def compute_score(line):
    ans = 0
    cnt = 0
    for ch in line:
        if ch == '{':
            cnt += 1
        elif ch == '}':
            ans += cnt
            cnt -= 1
    return ans
with open('inputs/day9.txt', 'r') as f:
    line = f.read()
    line = parse_line(line)
    print(compute_score(line))

6569
14212


# Day 10: Knot Hash

In [ ]:
with open('inputs/day10.txt', 'r') as f:
    moves = f.read().split(',')
# lst = [0, 1, 2, 3, 4]
# moves = [3, 4, 1, 5]
lst = list(range(256))
cur = 0
n = len(lst)
for skip, move in enumerate(map(int, moves)):
    temp = lst[cur:min(n, cur + move)] + lst[0:max(0, move - (n - cur))]
    for i, ch in enumerate(reversed(temp), cur):
        lst[i % n] = ch
    cur = (cur + move + skip) % n
print(lst[0] * lst[1])

In [ ]:
with open('inputs/day10.txt', 'r') as f:
    moves = []
    for ch in f.read():
        moves.append(ord(ch))
moves.extend([17, 31, 73, 47, 23])
def knot_hash(moves):
    lst = list(range(256))
    cur = 0
    n = len(lst)
    skip = 0
    for _ in range(64):
        for move in map(int, moves):
            temp = lst[cur:min(n, cur + move)] + lst[0:max(0, move - (n - cur))]
            for i, ch in enumerate(reversed(temp), cur):
                lst[i % n] = ch
            cur = (cur + move + skip) % n
            skip += 1
    code = []
    for i in range(0, 256, 16):
        code.append(reduce(ixor, lst[i:i+16]))
    ans = ""
    for num in code:
        ans += format(num, '02x')
    return ans
ans = knot_hash(moves)
print(ans)
print(len(ans))

# Day 11: Hex Ed

In [ ]:
with open('inputs/day11.txt', 'r') as f:
    lst = f.read().split(',')
counter = Counter(lst)

In [ ]:
x, y = 0, 0
for key, cnt in counter.items():
    if len(key) == 1:
        if 'n' in key or 's' in key:
            y += cnt if 'n' in key else -cnt        
    else:
        if 'n' in key or 's' in key:
            y += 0.5 * cnt if 'n' in key else -0.5 * cnt
        if 'e' in key or 'w' in key:
            x += cnt if 'e' in key else -cnt
print(round(abs(x) + abs((y - x // 2))))

In [ ]:
x, y = 0, 0
ans = 0
for key in lst:
    if len(key) == 1:
        if 'n' in key or 's' in key:
            y += 1 if 'n' in key else -1        
    else:
        if 'n' in key or 's' in key:
            y += 0.5 if 'n' in key else -0.5
        if 'e' in key or 'w' in key:
            x += 1 if 'e' in key else -1
    ans = max(ans, abs(x) + abs((y - x // 2)))  

In [ ]:
print(round(ans))

# Day 12: Digital Plumber

In [ ]:
graph = defaultdict(list)
nodes = set()
with open('inputs/day12.txt', 'r') as f:
    for line in f:
        line = line.strip()
        node, rest = line.split(' <-> ')
        nodes.add(node)
        for nei in rest.split(', '):
            nodes.add(nei)
            graph[node].append(nei)
            graph[nei].append(node)
visited = set()
def dfs(node):
    stack = [node]
    visited.add(node)
    cnt = 0
    while stack:
        node = stack.pop()
        cnt += 1
        for nei in graph[node]:
            if nei not in visited:
                visited.add(nei)
                stack.append(nei)
    return cnt
print(dfs('0'))
visited = set()
cnt = 0
for node in nodes:
    if node not in visited:
        cnt += 1
        dfs(node)
print(cnt)

# Day 13: Packet Scanners

In [ ]:
guards = {}
with open('inputs/day13.txt', 'r') as f:
    for line in f:
        depth, r = line.split(': ')
        depth, r = int(depth), int(r)
        guards[depth] = r
# print(guards)
N = max(guards) + 1
ranges = [guards.get(i, 0) for i in range(N)]
ps = [0] * N
vs = [1] * N
cur = 0
ans = 0
def update_guards(ps, vs):
    can_pass = True
    for i in range(len(ps)):
        if i in guards:
            if ps[i] == 0:
                can_pass = False
            ps[i] += vs[i]
            if ps[i] == 0 or ps[i] == ranges[i] - 1:
                vs[i] *= -1
    return can_pass
while cur < N:
    if cur in guards and 0 == ps[cur]:
        ans += cur * ranges[cur]
    update_guards(ps, vs)
    cur += 1
print(ans)

In [ ]:
ps = [0] * N
vs = [1] * N
print(N)
for i in range(N):
    for _ in range(i):
        ps[i] += vs[i]
        if ps[i] == 0 or ps[i] == ranges[i] - 1:
            vs[i] *= -1
delay = 0
while not update_guards(ps, vs):
    delay += 1
print(delay)

In [ ]:
# from array import array
# def can_pass(ps, vs):
#     cur = 0
#     while cur < N:
#         if cur in guards and 0 == ps[cur]:
#             return False
#         update_guards(ps, vs)
#         cur += 1
#     return True
# ps = array('i', [0] * N)
# vs = array('i', [1] * N)
# delay = 0
# while True:
#     if can_pass(ps[:], vs[:]):
#         print(delay)
#         break
#     delay += 1
#     update_guards(ps, vs)

In [ ]:
# 3823370

# Day 14: Disk Defragmentation

In [ ]:
# hfdlxzhv
# s = "flqrgnkx"
s = "hfdlxzhv"
def disk(s):
    grid = []
    for i in range(128):
        moves = list(map(ord, '{}-{}'.format(s, i)))
        moves.extend([17, 31, 73, 47, 23])
        ans = knot_hash(moves)
        row = ""
        for j in range(32):
            row += '{:04b}'.format(int(ans[j], 16))
        grid.append(row)
    return [list(row) for row in grid]
grid = disk(s)
print(sum(item == '1' for row in grid
                      for item in row))

In [ ]:
# number of connected components
def count_components(grid):
    m, n = len(grid), len(grid[0])
    def dfs(r, c):
        stack = [(r, c)]
        while stack:
            i, j = stack.pop()
            for newi, newj in (i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1):
                if 0 <= newi < len(grid) and 0 <= newj < len(grid[0]) and grid[newi][newj] == '1':
                    stack.append((newi, newj))
                    grid[newi][newj] = '#'
    cnt = 0
    for i, row in enumerate(grid):
        for j, ch in enumerate(row):
            if ch == '1':
                dfs(i, j)
                cnt += 1
    return(cnt)
print(count_components(grid))

# Day 15: Dueling Generators
Generator A starts with 516
Generator B starts with 190

In [ ]:
def duel_generator(A, B):
    return A * 16807 % 2147483647, B * 48271 % 2147483647

In [ ]:
cnt = 0
A, B = 516, 190
sixteen = 2 ** 16
for i in range(40_000_000):
    A, B = duel_generator(A, B)
    cnt += (A % sixteen) == (B % sixteen)
print(cnt)

In [ ]:
def duel_generator_new(A, B):
    A, B = A * 16807 % 2147483647,  B * 48271 % 2147483647
    while A % 4 != 0:
        A = A * 16807 % 2147483647
    while B % 8 != 0:
        B = B * 48271 % 2147483647
    return A, B
cnt = 0
A, B = 516, 190
sixteen = 2 ** 16
for i in range(5_000_000):
    A, B = duel_generator_new(A, B)
    cnt += (A % sixteen) == (B % sixteen)
print(cnt)

# Day 16: Permutation Promenade

In [ ]:
indexes = {}
n = 16
class Node:
    def __init__(self, val):
        self.val = val
lst = [Node(chr(i)) for i in range(97, 97 + n)]
for i, node in enumerate(lst):
    indexes[node.val] = i

In [ ]:
with open('inputs/day16.txt', 'r') as f:
    lines = f.read().strip().split(',')
    ops = []
    for line in lines:
        if line[0] == 's':
            shift = int(line[1:]) % n
            ops.append(('s', shift))
        elif line[0] == 'x':
            a, b = map(int, line[1:].split('/'))
            ops.append(('x', a, b))
        else:
            a, b = line[1:].split('/')
            ops.append(('p', a, b))
    
def dance(ops, cur):
    for op in ops:
        if op[0] == 's':
            shift = op[1]
            cur -= shift
            if cur < 0:
                cur += n
        elif op[0] == 'x':
            a, b = op[1:]
            a, b = (cur + a) % n, (cur + b) % n
            A, B = lst[a].val, lst[b].val
            lst[a].val, lst[b].val = lst[b].val, lst[a].val
            indexes[A], indexes[B] = indexes[B], indexes[A]
        else:
            a, b = op[1:]
            ia, ib = indexes[a], indexes[b]
            lst[ia].val, lst[ib].val = b, a
            indexes[a], indexes[b] = indexes[b], indexes[a]
    return cur

In [ ]:
def show_position(lst, cur):
    for i in range(n):
        print(lst[(cur + i) % n].val, end = '')
    print()

In [ ]:
cur = 0
cur = dance(ops, cur)
show_position(lst, cur)

In [ ]:
lst = [Node(chr(i)) for i in range(97, 97 + n)]
for i, node in enumerate(lst):
    indexes[node.val] = i
cur = 0
old = [lst[(cur + i) % n].val for i in range(n)]
for i in range(1_000_000_000):
    cur = dance(ops, cur)
    if all(old[i] == lst[(cur + i) % n].val for i in range(n)):
        idx = i + 1
        break

In [ ]:
lst = [Node(chr(i)) for i in range(97, 97 + n)]
for i, node in enumerate(lst):
    indexes[node.val] = i
cur = 0
for i in range(1_000_000_000 % 30):
    cur = dance(ops, cur)
show_position(lst, cur)

# Day 17: Spinlock

In [ ]:
class Node:
    def __init__(self, val):
        self.val = val
        self.next = None
head = Node(0)
head.next = head
def print_linkedlist(cur, n):
    for i in range(n):
        print(cur.val, end=', ')
        cur = cur.next
    print()
cur = head
# print_linkedlist(head, 1)
steps = 314
end = 2017
for num in range(1, end + 1):
    node = Node(num)
    for _ in range(steps):
        cur = cur.next
    node.next = cur.next
    cur.next = node
    cur = node
#     print_linkedlist(head, num + 1)
print(cur.next.val)
# print(head.next.val)
# only need to know the node after head
end = 50000000
cur = 0
ans = None
step = 314
for length in range(1, end + 1):
    insert = (cur + step) % length
    if insert == 0:
        ans = length
    cur = insert + 1
print(ans)

# Day 18: Duet

In [ ]:
import operator
registers = defaultdict(int)
def fetch(s, idx=0):
    if s.isalpha():
        return registers.setdefault(s, idx)
    else:
        return int(s)
        
sound = None
funcs = {'add' : operator.add, 'mul' : operator.mul, 'mod' : operator.mod}
with open('inputs/day18.txt', 'r') as f:
    ops = []
    for line in f:
        op, *hs = line.strip().split()
        ops.append((op, hs))
i = 0
while i < len(ops):
    op, hs = ops[i]
    if op == 'set':
        var, val = hs
        val = fetch(val)
        registers[var] = val
    elif op == 'jgz':
        var, val = hs
        val = fetch(val)
        var_val = fetch(var)
        if var_val > 0:
            i += val
            continue
    elif op == 'snd':
        sound = registers[hs[0]]
    elif op == 'rcv':
        var = hs[0]
        if registers[var] != 0 and sound is not None:
            registers[val] = sound
            print(sound)
            break
    else:
        var, val = hs
        val = fetch(val)
        var_val = fetch(var)
        registers[var] = funcs[op](var_val, val)
    i += 1

In [ ]:
import copy
with open('inputs/day18.txt', 'r') as f:
    ops = []
    for line in f:
        op, *hs = line.strip().split()
        ops.append((op, hs))
registers = [defaultdict(int), defaultdict(int)]
def fetch(s, idx):
    if s.isalpha():
        if s == 'p':
            return registers[idx].setdefault('p', idx)
        else:
            return registers[idx][s]
    else:
        return int(s)
msgs = [deque(), deque()]
curs = [0, 0]
cnt = 0
step = 0
while True:
    step += 1
#     print(ops[curs[1]])
    terminate = True
#     print(ops[curs[0]], ops[curs[1]])
    for i in range(2): # ith program
        if curs[i] >= len(ops):
            continue
        op, hs = ops[curs[i]]
        if op == 'set':
            var, val = hs
            val = fetch(val, i)
            registers[i][var] = val
            terminate = False
        elif op == 'jgz':
            var, val = hs
            val = fetch(val, i)
            var_val = fetch(var, i)
            terminate = False
            if var_val > 0:
                curs[i] += val
                continue
        elif op == 'snd':
            val = fetch(hs[0], i)
            msgs[1 - i].append(val)
            if i == 1:
                cnt += 1
            terminate = False
        elif op == 'rcv':
            var = hs[0]
            if msgs[i]:
                terminate = False
                registers[i][var] = msgs[i].popleft()
            else:
                continue
        else:
            var, val = hs
            val = fetch(val, i)
            var_val = fetch(var, i)
            registers[i][var] = funcs[op](var_val, val)
            terminate = False
        curs[i] += 1
    if terminate:
        break
print(cnt)

# Day 19: A Series of Tubes

In [ ]:
tubes = {}
x, y = 0, 0
with open('inputs/day19.txt', 'r') as f:
    for i, line in enumerate(f):
        line = line.rstrip('\n')
        for j, ch in enumerate(line):
            tubes[i, j] = ch
            if i == 0 and ch == '|':
                y = j
assert(tubes[x, y] == '|')

In [ ]:
def next_move(tubes, i, j, pre_i, pre_j):
    if tubes[i, j] in ('|', '-'):
        newi, newj = 2 * i - pre_i, 2 * j - pre_j
        if (newi, newj) in tubes:
            return newi, newj
        else:
            return None, None
    for di, dj in (-1, 0), (1, 0), (0, 1), (0, -1):
        newi, newj = i + di, j + dj
        if (newi, newj) == (pre_i, pre_j):
            continue
        if (newi, newj) in tubes and tubes[newi, newj] != ' ':
            return newi, newj
    return (None, None)
dx, dy = 1, 0
pre_x, pre_y = x - dx, y - dy
ans = []
cnt = 0
while True:
    cnt += 1
    if tubes[x, y].isalpha():
        ans.append(tubes[x, y])
    newx, newy = next_move(tubes, x, y, pre_x, pre_y)
    if newx == None:
        break
    pre_x, pre_y = x, y
    x, y = newx, newy
print(''.join(ans))
print(cnt)

# Day 20: Particle Swarm

In [ ]:
# p=<-4897,3080,2133>, v=<-58,-15,-78>, a=<17,-7,0>
raw_ps, raw_vs, raw_acs = [], [], []
def parse_helper(s):
    return list(map(int, s[1:-1].split(',')))
with open('inputs/day20.txt', 'r') as f:
    for line in f:
        items = re.findall('\<[0123456789\-,]+\>', line)
        raw_ps.append(parse_helper(items[0]))
        raw_vs.append(parse_helper(items[1]))
        raw_acs.append(parse_helper(items[2]))
raw_ps = np.array(raw_ps, dtype=np.int64)
raw_vs = np.array(raw_vs, dtype=np.int64)
raw_acs = np.array(raw_acs, dtype=np.int64)
print(raw_ps.shape)

In [ ]:
ps, vs, acs = raw_ps.copy(), raw_vs.copy(), raw_acs.copy()
def closest_particle(ps):
    return min(enumerate(ps), key=lambda x : np.abs(x[1]).sum())
unchanged_steps = 0
min_idx, min_distance = closest_particle(ps)
while unchanged_steps < 500: # min_idx does not change for 500 steps
    vs += acs
    ps += vs
    idx, distance = closest_particle(ps)
    if idx != min_idx:
        min_idx = idx
        unchanged_steps = 0
    else:
        unchanged_steps += 1
print(min_idx)

In [ ]:
ps, vs, acs = raw_ps.copy(), raw_vs.copy(), raw_acs.copy()
def find_collisions(ps, vs, acs):
    d = {}
    keeps = [True] * ps.shape[0]
    for i, row in enumerate(map(tuple, ps)):
        if row in d:
            keeps[d[row]] = False
            keeps[i] = False
        else:
            d[row] = i
    find_collide = not all(keeps)
    if find_collide:
        ps = ps[keeps]
        vs = vs[keeps]
        acs = acs[keeps]
    return ps, vs, acs, find_collide

no_collide_steps = 0
while no_collide_steps < 500:
    vs += acs
    ps += vs
    ps, vs, acs, find_collide = find_collisions(ps, vs, acs)
    if find_collide:
        no_collide_steps = 0
    else:
        no_collide_steps += 1
print(ps.shape[0])

# Day 21: Fractal Art

In [ ]:
rules = {}
def str_to_matrix(s):
    s = s.strip()
    n = s.count('/') + 1
    m = np.zeros((n, n), dtype=np.int8)
    for i, row in enumerate(s.split('/')):
        for j, ch in enumerate(row):
            if ch == '#':
                m[i, j] = 1
    return m
def parse(rules, line):
    lm, rm = map(str_to_matrix, line.split(' => '))
    n = lm.shape[0]
    for temp in (lm, np.fliplr(lm), np.flipud(lm)):
        rules[temp.tostring()] = rm
        rules[np.rot90(temp).tostring()] = rm
        rules[np.rot90(temp, 2).tostring()] = rm
        rules[np.rot90(temp, 3).tostring()] = rm
with open('inputs/day21.txt', 'r') as f:
    for line in f:
        parse(rules, line)
# print(rules)

In [ ]:
len(rules) == (2**4 + 2**9)

In [ ]:
grid = np.array([[0, 1, 0], [0, 0, 1], [1, 1, 1]], dtype=np.int8)
for i in range(18):
    n = len(grid)
    if n % 2 == 0:
        m = n // 2 * 3
        size = 2
    else:
        assert(n % 3 == 0)
        m = n // 3 * 4
        size = 3
    new_grid = np.zeros((m, m), dtype=np.int8)
    new_size = size + 1
    for i in range(0, n, size):
        for j in range(0, n, size):
            newi, newj = i // size * new_size, j // size * new_size
            new_grid[newi : newi + new_size, newj : newj + new_size] = rules[grid[i:i+size,j:j+size].tostring()]
    grid = new_grid
print(grid.sum())

# Day 22: Sporifica Virus

In [ ]:
infected_ps = set()
with open('inputs/day22.txt', 'r') as f:
    lines = [line.rstrip('\n') for line in f]
#     print(lines)
    m, n = len(lines), len(lines[0])
    for i, line in enumerate(lines):
        for j, ch in enumerate(line):
            if ch == '#':
                infected_ps.add((i, j))
dirs = [(-1, 0), (0, 1), (1, 0), (0, -1)]
x, y = m // 2, n // 2
dir_idx = 0
cnt = 0
for _ in range(10000):
    # infected: turn right
    dir_idx = dir_idx + 1 if (x, y) in infected_ps else dir_idx - 1
    dir_idx %= 4
    assert(0 <= dir_idx < 4)
    dx, dy = dirs[dir_idx]
    if (x, y) in infected_ps:
        infected_ps.remove((x, y))
    else:
        infected_ps.add((x, y))
        cnt += 1
    x, y = x + dx, y + dy
print(cnt)

- Clean nodes become weakened.
- Weakened nodes become infected.
- Infected nodes become flagged.
- Flagged nodes become clean.



1. Decide which way to turn based on the current node:
    - If it is clean, it turns left.
    - If it is weakened, it does not turn, and will continue moving in the same direction.
    - If it is infected, it turns right.
    - If it is flagged, it reverses direction, and will go back the way it came.
2. Modify the state of the current node, as described above.
3. The virus carrier moves forward one node in the direction it is facing.

In [ ]:
evolves = {'c' : 'w', 'w' : 'i', 'i' : 'f', 'f' : 'c'}
states = {}
with open('inputs/day22.txt', 'r') as f:
    lines = [line.rstrip('\n') for line in f]
    m, n = len(lines), len(lines[0])
    for i, line in enumerate(lines):
        for j, ch in enumerate(line):
            if ch == '#':
                states[(i, j)] = 'i'
            elif ch == '.':
                states[(i, j)] = 'c'
dirs = [(-1, 0), (0, 1), (1, 0), (0, -1)]
x, y = m // 2, n // 2
dir_idx = 0
cnt = 0
for _ in range(10000000):
    if (x, y) not in states or states[x, y] == 'c':
        dir_idx = dir_idx - 1
    elif states[x, y] == 'i':
        dir_idx = dir_idx + 1
    elif states[x, y] == 'f':
        dir_idx = dir_idx + 2
    dir_idx = dir_idx % 4
    nxt_state = 'w' if (x, y) not in states else evolves[states[x, y]]
    cnt += nxt_state == 'i'
    states[x, y] = nxt_state
    dx, dy = dirs[dir_idx]
    x, y = x + dx, y + dy
print(cnt)

# Day 23: Coprocessor Conflagration

In [ ]:
with open('inputs/day23.txt', 'r') as f:
    ops = []
    for line in f:
        op, a, b = line.strip().split()
        ops.append((op, a, b))
def fetch_value(registers, s):
    if s in registers:
        return registers[s]
    else:
        return int(s)
funcs = {'sub' : operator.sub, 'mul' : operator.mul}
def run_program(registers):
    """
    run thr program and count number muls
    """
    i = 0
    cnt = 0
    while i < len(ops):
        op, a, b = ops[i]
        if op == 'mul':
            cnt += 1
        if op == 'set':
            registers[a] = fetch_value(registers, b)
        elif op == 'jnz':
            x = fetch_value(registers, a)
            if x != 0:
                y = fetch_value(registers, b)
                i += y
                continue
        else:
            x = fetch_value(registers, a)
            y = fetch_value(registers, b)
            registers[a] = funcs[op](x, y)
        i += 1
    return cnt
registers = dict(zip('abcdefgh', [0] * 8))
print(run_program(registers))
# needs to change
# registers = dict(zip('abcdefgh', [1] + [0] * 7))
# run_program(registers)
print(registers['h'])

In [ ]:
# part 2 set a = 1
cnt = 0
for x in range(109300, 126300 + 1, 17):
    for i in range(2, x):
        if x % i == 0:
            cnt += 1
            break
print(cnt)

# Day 24: Electromagnetic Moat

In [ ]:
graph = defaultdict(set)
nodes = set()
with open('inputs/day24.txt', 'r') as f:
    for line in f:
        line = line.strip()
        a, b = map(int, line.split('/'))
        graph[a].add(b)
        graph[b].add(a)
        nodes.add(frozenset({a, b}))

In [ ]:
nodes_back = nodes.copy()

In [ ]:
ans = [0]
def dfs(cur, acc, ans):
    for nei in graph[cur]:
        item = frozenset({nei, cur})
        if item in nodes:
            nodes.remove(item)
            dfs(nei, acc + 2 * nei , ans)
            nodes.add(item)
    if acc - cur > ans[0]:
        ans[0] = acc - cur
dfs(0, 0, ans)
print(ans[0])

In [ ]:
ans = [0, 0]
def dfs(cur, length, acc, ans):
    for nei in graph[cur]:
        item = frozenset({nei, cur})
        if item in nodes:
            nodes.remove(item)
            dfs(nei, length + 1, acc + 2 * nei, ans)
            nodes.add(item)
    if length > ans[1] or (length == ans[1] and acc - cur > ans[0]):
        ans[0] = acc - cur
        ans[1] = length
dfs(0, 0, 0, ans)
print(ans)

# Day 25: The Halting Problem 

In [ ]:
rules = {'A' : ((1, +1, 'B'), (1, -1, 'E')),
         'B' : ((1, +1, 'C'), (1, +1, 'F')),
         'C' : ((1, -1, 'D'), (0, +1, 'B')),
         'D' : ((1, +1, 'E'), (0, -1, 'C')),
         'E' : ((1, -1, 'A'), (0, +1, 'D')),
         'F' : ((1, +1, 'A'), (1, +1, 'C')),
        }

In [ ]:
tape = deque([0] * 4000)
cur = 2000
nsteps = 12459852
state = 'A'
for _ in range(nsteps):
    cur_val = tape[cur]
    write, move, state = rules[state][cur_val]
    tape[cur] = write
    cur += move
    if cur == len(tape) and move == 1:
        tape.append(0)
    if cur == -1 and move == -1:
        tape.appendleft(0)
        cur = 0
print(sum(tape))